### Segment Clustering neighborhoods of toronto

#### import packages

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import geocoder

import urllib
import folium
from geopy.geocoders import Nominatim

#### Define URL from where to get the data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#with open(url) as html_file:
#    soup = BeautifulSoup(html_file,"lxml")
    
#print(soup)
headers = {'User-Agent': 'Mozilla/5.0'}



#### Fetch the data from url

In [3]:
response = urllib.request.urlopen(url)
rawhtml = response.read()

#### add fetched data to soup object

In [4]:
soup = BeautifulSoup(rawhtml,"html.parser")
    
#print(soup.select('table'))

#### Get the table and clean it

In [5]:
pclist = []
prevPC = "" # stores the previous postal code to see if we need to append neighbourhoods or create a new row
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
           #skip row
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b; # set the neighbourhood equal to the borough
            
            if (pc == prevPC):
                # append neighbourhoods into comma delimited string
                currNeighbourhoods = currNeighbourhoods + ", " + n
                 
            else:
                # new postal code, so write out previous row
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc # set a new PrevPC value for next loop
                prevBorough = b
                currNeighbourhoods = n # start a new neighbourhoods string
# write out last row
pclist.append([prevPC,prevBorough,currNeighbourhoods])               
            
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### get shape of table

In [6]:
toronto_df.shape

(103, 3)

#### Get latitude and longitude of each postal code

In [ ]:
'''
# !pip install geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

lat_lng_coords = None
# create lists to store our new lats and longs
lats = []
longs=[]
#loop through our dataframe and look up the lat/long of each postal code
for index, row in toronto_df.iterrows():
    #print (row["PostalCode"])
    postal_code=row["PostalCode"]
    geolocator = Nominatim(user_agent="foursquare_agent")
    # loop until you get the coordinates
    location = None
    while(location is None):
        
        location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code), timeout=100)
    latitude = location.latitude
    longitude = location.longitude
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        #lat_lng_coords = g.latlng

    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    lats.append(latitude)
    longs.append(longitude)
    #print ('postal code {} had latitude {} and longitude {}'.format(postal_code, latitude, longitude))

toronto_df['Latitude'] = lats
toronto_df['Longitude'] = longs
toronto_df
'''

In [8]:
## After trying a lot with above commented code ,
## I got the code from the discussion forum
# "https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/BNqlp1qxEemUkAqlaf0TXA/replies/2pD_ZVzMEemXoQpaMIA2fg/comments/pi-GDnCzEem6JBJNWiPf3A


Lat_list=[]
Lng_list=[]
for i in range(toronto_df.shape[0]):
    address='{}, Toronto, Ontario'.format(toronto_df.at[i,'PostalCode'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [9]:
toronto_df['Latitude'] = Lat_list
toronto_df['Longitude'] = Lng_list
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
5,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749055,-79.362227
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180


In [10]:
toronto_df.shape

(103, 5)

## Assignment 3 Segmenting and clustering

In [3]:
#toronto_df.to_csv("toronto_df.csv", index = False)
toronto_df = pd.read_csv("toronto_df.csv")

In [1]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:


map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



In [9]:
radius=500
LIMIT=100

In [15]:
def getNearVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    for name , lat, lng in zip (names, latitudes, longitudes):
                # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood_Lat', 
                  'Neighborhood_Long', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

### get all the venues and its categories 

In [16]:
toronto_venues = getNearVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

#### check size of venues

In [17]:
toronto_venues.head(10)

,Neighborhood,Neighborhood_Lat,Neighborhood_Long,Venue,Venue_Lat,Venue_Long,Venue_Category
0,Parkwoods,43.752420,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752420,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.730600,-79.313265,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.730600,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
5,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
6,Harbourfront,43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
7,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
8,Harbourfront,43.650295,-79.359166,Cacao 70,43.650067,-79.360723,Dessert Shop
9,Harbourfront,43.650295,-79.359166,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue


In [18]:
toronto_venues.shape

(2460, 7)

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood_Lat,Neighborhood_Long,Venue,Venue_Lat,Venue_Long,Venue_Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,63,63,63,63,63,63
"Birch Cliff, Cliffside West",7,7,7,7,7,7


#### get one hot dataframe

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### get shape of new dataframe

In [22]:
toronto_onehot.shape

(2460, 261)

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.00,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.0,...,0.0,0.010000,0.00,0.000000,0.000000,0.010000,0.000000,0.0,0.01,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.071429,0.000000,0.000000,0.0,0.00,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.0,0.00,0.000000
4,"Alderwood, Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
5,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
6,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
7,Berczy Park,0.000000,0.00,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.0,...,0.0,0.015873,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
8,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000


#### get 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4   Restaurant  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.14
1  Chinese Restaurant  0.07
2     Bubble Tea Shop  0.07
3       Grocery Store  0.07
4        Skating Rink  0.07


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                     venue  freq
0                 Pharmacy   0.5
1         Sushi Restaurant   0.5
2      Monument / Landmark   0.0
3                Nightclub   0.0
4  New American Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.13
1            Beer Store  0.07
2              Pharmacy  0.07
3  Caribbean Restaurant  0.07
4          Liquor Store  0.07


----Alderwood, Long Branch----
                venue  freq
0                 Pub  0.25

----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.16
1        Hotel  0.07
2         Café  0.07
3   Restaurant  0.04
4          Gym  0.03


----Flemingdon Park, Don Mills South----
             venue  freq
0      Coffee Shop  0.14
1       Beer Store  0.14
2    Grocery Store  0.14
3     Intersection  0.14
4  Bubble Tea Shop  0.14


----Forest Hill North, Forest Hill West----
               venue  freq
0       Home Service   0.5
1               Park   0.5
2  Afghan Restaurant   0.0
3             Museum   0.0
4       Noodle House   0.0


----Glencairn----
                       venue  freq
0                Pizza Place  0.23
1   Mediterranean Restaurant  0.08
2                Gas Station  0.08
3  Latin American Restaurant  0.08
4             Sandwich Place  0.08


----Guildwood, Morningside, West Hill----
                        venue  freq
0  Construction & Landscaping  0.33
1                        Park  0.33
2        Gym / Fitness Center  0.33
3       

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Burger Joint,Breakfast Spot,American Restaurant,Gym,Asian Restaurant,Japanese Restaurant
1,Agincourt,Shopping Mall,Department Store,Skating Rink,Sushi Restaurant,Supermarket,Bakery,Shanghai Restaurant,Grocery Store,Chinese Restaurant,Pool
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Sushi Restaurant,Pharmacy,Yoga Studio,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Fried Chicken Joint,Liquor Store,Park,Fast Food Restaurant
4,"Alderwood, Long Branch",Convenience Store,Pub,Gym,Athletics & Sports,Field,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### Cluster Neighborhoods

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 1, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 2, 4, 4, 4, 1, 1, 4, 2, 4, 4, 1, 4, 4, 2, 4, 2, 1, 4, 4, 4, 2,
       4, 2, 4, 4, 2, 2, 3, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 0, 2, 4, 2,
       4, 4, 4, 2, 4, 2, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 2, 4, 4,
       4, 4, 1, 4, 4, 2])

In [32]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,2.0,Food & Drink Shop,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
1,M4A,North York,Victoria Village,43.730600,-79.313265,2.0,Park,Grocery Store,Yoga Studio,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Electronics Store
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,4.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Historic Site,Pub,Ice Cream Shop,Gastropub,Breakfast Spot,Brewery
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286,4.0,Clothing Store,Sushi Restaurant,Food Court,American Restaurant,Cosmetics Shop,Men's Store,Furniture / Home Store,Toy / Game Store,Coffee Shop,Bookstore
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715,4.0,Coffee Shop,Café,Pharmacy,Sandwich Place,Gym,Park,Italian Restaurant,Burrito Place,Fried Chicken Joint,Sushi Restaurant


#### Drop NA rows

In [33]:
toronto_merged=toronto_merged.dropna()

In [34]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

#### plot clusters on map

In [36]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster 1 

In [37]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,0,Convenience Store,Auto Garage,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish Market,Ethiopian Restaurant


### cluster 2

In [38]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,1,Home Service,Construction & Landscaping,Hockey Arena,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
22,Scarborough,1,Construction & Landscaping,Coffee Shop,Business Service,Park,Yoga Studio,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
40,North York,1,Food Court,Airport,Coffee Shop,Park,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
50,North York,1,Construction & Landscaping,Italian Restaurant,Rental Car Location,Home Service,Historic Site,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market
53,North York,1,Home Service,Construction & Landscaping,Business Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Ethiopian Restaurant,Fish Market
56,York,1,Construction & Landscaping,Playground,Coffee Shop,Fast Food Restaurant,Yoga Studio,Field,Event Space,Falafel Restaurant,Farm,Farmers Market
57,North York,1,Coffee Shop,Nightclub,Park,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,Ethiopian Restaurant
86,Central Toronto,1,Coffee Shop,Light Rail Station,Liquor Store,Supermarket,Yoga Studio,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Field


### cluster 3

In [39]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Food & Drink Shop,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
1,North York,2,Park,Grocery Store,Yoga Studio,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Electronics Store
7,North York,2,Burger Joint,Soccer Field,Gas Station,Park,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
11,Etobicoke,2,Park,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field
18,Scarborough,2,Construction & Landscaping,Park,Gym / Fitness Center,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Electronics Store
21,York,2,Park,Market,Mexican Restaurant,Fast Food Restaurant,Bakery,Beer Store,Gym,Sporting Goods Shop,Women's Store,Flower Shop
35,East York,2,Bar,Music Venue,Italian Restaurant,Park,Farmers Market,Yoga Studio,Ethiopian Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant
36,Downtown Toronto,2,Pier,Harbor / Marina,Thrift / Vintage Store,Park,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
39,North York,2,Construction & Landscaping,Golf Driving Range,Park,Trail,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
41,East Toronto,2,Discount Store,Bus Line,Grocery Store,Park,Yoga Studio,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


### cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,3,Bar,Ethiopian Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


### cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Bakery,Theater,Boat or Ferry,Historic Site,Pub,Ice Cream Shop,Gastropub,Breakfast Spot,Brewery
3,North York,4,Clothing Store,Sushi Restaurant,Food Court,American Restaurant,Cosmetics Shop,Men's Store,Furniture / Home Store,Toy / Game Store,Coffee Shop,Bookstore
4,Downtown Toronto,4,Coffee Shop,Café,Pharmacy,Sandwich Place,Gym,Park,Italian Restaurant,Burrito Place,Fried Chicken Joint,Sushi Restaurant
5,Queen's Park,4,Coffee Shop,Café,Pharmacy,Sandwich Place,Gym,Park,Italian Restaurant,Burrito Place,Fried Chicken Joint,Sushi Restaurant
8,East York,4,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Café,Breakfast Spot,Gastropub,Rock Climbing Spot,Bank,Intersection,Bus Line
9,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Fast Food Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store,Restaurant,Bookstore
10,North York,4,Pizza Place,Mediterranean Restaurant,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Latin American Restaurant,Fast Food Restaurant,Grocery Store,Ice Cream Shop,Asian Restaurant
13,North York,4,Bubble Tea Shop,Intersection,Coffee Shop,Grocery Store,Gym,Supermarket,Beer Store,Falafel Restaurant,Farm,Farmers Market
14,East York,4,Grocery Store,Bus Line,Dance Studio,Metro Station,Middle Eastern Restaurant,Café,Breakfast Spot,Gas Station,Bar,Bakery
15,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Bakery,Breakfast Spot,Hotel,Beer Bar,Cocktail Bar
